# scPrinter setup for Bos taurus
* Our goal? Being to characterize differential TF profiles between peak and late lactation

In [1]:
import torch
torch.cuda.is_available()
import cupy
a = cupy.zeros((1000, 1000)) # check gpu usage

In [2]:
a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Comparing TF Profiles between Peak and Late Lactation

In [3]:
%load_ext autoreload
%autoreload 2
import scprinter as scp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import os
import pickle
import torch
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from scanpy.plotting.palettes import zeileis_28
from tqdm.contrib.concurrent import *
from tqdm.auto import *
import anndata
import scanpy as sc
import json
import csv
import re
from sklearn.preprocessing import OneHotEncoder

/net/talisker/home/benos/mae117/.conda/envs/the-milk-man/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/net/talisker/home/benos/mae117/.conda/envs/the-milk-man/lib/python3.11/site-packages/norns/cfg.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/net/talisker/home/benos/mae117/.conda/envs/the-milk-man/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be 

In [ ]:
work_dir = "/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man"
ref_genome="/net/talisker/home/benos/mae117/.local/share/genomes/ARS-UCD2.0/ARS-UCD2.0.fa"

In [ ]:
# 1) Build a custom genome object once (FASTA -> twoBit -> bias bigWig per docs)
# See: "Tutorial using custom genome"
bovine = scp.genome.create_custom_genome(
    fasta_path="Bos_taurus.ARS-UCD1.2.dna.primary_assembly.fa",
    genome_name="bos_taurus_ARSUCD12",
    work_dir=work_dir,
    overwrite=False
)
# This step creates and caches the bias model bigWig for Tn5 and chrom sizes.

# 2) List your fragment files & barcodes (barcodes are just sample names here)
frag_files = [
    "BovineCtrl_01.fragments.tsv.gz",
    "BovineCtrl_02.fragments.tsv.gz",
    # ...
]
barcodes = [None]*len(frag_files) # scPrinter will take the 4th col as barcode=sample

printer = scp.pp.import_fragments(
    path_to_frags=frag_files,
    barcodes=barcodes,
    savename=f"{work_dir}/bovine_controls_scprinter.h5ad",
    genome=bovine,         # <-- custom genome object
    min_num_fragments=1_000, 
    min_tsse=7,
    sorted_by_barcode=False,
    low_memory=False,
)